# 09 — Composing Chains (Using One Chain's Output as Another's Input)

In LCEL, you can use the **output of one chain as the input of another**. This lets you build pipelines where one sub-problem feeds the next (e.g., *politician → country* and then *country → continent*).

You compose chains with the `|` operator and you can place **subchains** inside parallel dict blocks to fan-out values that will later be fanned-in by a prompt.

In [1]:
# ╔══════════════════════════════════════════════════════╗
# ║ Setup: Load environment variables & initialize model ║
# ╚══════════════════════════════════════════════════════╝

import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
# from langchain_groq import ChatGroq

chat_model = ChatOpenAI(model="gpt-4o-mini")
# chat_model = ChatGroq(model="llama-3.1-70b-versatile")

print("✅ Environment loaded and model ready.")

✅ Environment loaded and model ready.


## Example 1 — Any composed chain (context)

A composed chain is simply a sequence like **prompt → model → parser**:

```python
# Illustrative example (not executed here):
# composed_chain.invoke({"politician": "Attila"})
# → "Attila the Hun's conquests... rather than positive contributions to humanity."
```

Nothing new here: chains can be **composed in series** with `|`. The important part is that the **final output** can then be used by another chain.

## Example 2 — A chain inside another (dependent fan-in)

**Goal:**
1) Ask the **country** of a politician.
2) Using that country, ask the **continent**, in a given language.

We will build two subchains and nest the first one inside a parallel dict block that also passes the language through.

In [2]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt1 = ChatPromptTemplate.from_template(
    "What country is {politician} from?"
)
prompt2 = ChatPromptTemplate.from_template(
    "What continent is the country {country} in? Respond in {language}."
)

# Chain 1: (politician) -> country (as plain string)
chain1 = prompt1 | chat_model | StrOutputParser()

# Chain 2: use chain1's output as the 'country' variable and also pass 'language'
chain2 = (
    {
        "country": chain1,                 # subchain runs with the SAME input (needs 'politician')
        "language": itemgetter("language") # pass 'language' directly from the incoming dict
    }
    | prompt2
    | chat_model
    | StrOutputParser()
)

print(
    chain2.invoke({"politician": "François Mitterrand", "language": "French"})
)

Le pays dont François Mitterrand était originaire, la France, se trouve en Europe.


### What’s happening (step by step)

1. **Input:** `{"politician": "...", "language": "..."}`.
2. The parallel dict runs **both keys**:
   - `"country": chain1` → executes `chain1` with the **same input** (it uses `politician`) and returns a country string.
   - `"language": itemgetter("language")` → extracts `language` unchanged from the input.
3. A new dict is formed: `{"country": <country>, "language": <language>}` which feeds `prompt2`.
4. `prompt2 → model → parser` produce the final answer (in the requested language).

## Remember what `itemgetter` does

`itemgetter` (from Python’s `operator` module) returns a function that pulls a specific key from a dict — perfect as a tiny wiring tool inside chains.

```python
from operator import itemgetter
extract_language = itemgetter("language")
extract_language({"language": "French", "other": 123})  # → "French"
```

Use it whenever you want to **route** one field from the input to a chain stage **without modifying it**.

## Patterns to learn from this design

- **Subchains as nodes:** any runnable can be a value inside a dict; the **subchain’s output** becomes the value for that key.
- **`itemgetter` as a cable:** forwards fields from the input verbatim.
- **Consistent types:** if `prompt2` expects `country` as a **string**, ensure `chain1` outputs a **string** (hence the `StrOutputParser`).
- **Fan-out → Fan-in:** the dict creates a **fan-out** (country/language). The prompt then **fans in** (consumes) those variables.

## Best practices

- **Fix output types early** (e.g., `StrOutputParser`, JSON parsers) to avoid downstream mismatches.
- **Name dict keys clearly** to match the variables consumed later (`country`, `language`, etc.).
- **Reuse subchains** — define once, use many times.
- **Validate assumptions** — LLMs can return noisy strings (e.g., *"France (Europe)"*). If you need exactness, post-process (regex/normalization).

## Useful variants

- **With `.assign()`** — build a base dict and then **add** derived keys:

```python
from langchain_core.runnables import RunnableParallel, RunnableLambda

base = {"country": chain1, "language": itemgetter("language")}
vars_dict = RunnableParallel(base).assign(
    meta=RunnableLambda(lambda d: {"country_len": len(d["country"])})
)
final = vars_dict | prompt2 | chat_model | StrOutputParser()
```

- **With `RunnableParallel` explicitly** — when you want to emphasize the fan-out stage.
- **With `RunnableBranch`** — if the subchain depends on conditions (e.g., choose different prompts by language).